# Positive EV Sports Betting Calculator

### Dependencies

In [1]:
import numpy as np
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup 

In [2]:
class bet_arb():
    def __init__(self):
        pass
    
    def american_to_decimal(odds_list):
        decimal_list = []

        for odd in odds_list:
            if odd > 0:
                decimal_odd = round((odd/100) + 1,2)
                decimal_list.append(decimal_odd)
                
            elif odd < 0: 
                decimal_odd = round(1 - (100 /odd),2) 
                decimal_list.append(decimal_odd)
        return(decimal_list) 
    
    def draft_kings_webscrap(self):
        dk_url = "https://sportsbook.draftkings.com/leagues/mma/ufc"
        options = Options()
        options.headless = True
        driver = webdriver.Chrome(options = options, service=Service(ChromeDriverManager().install()))
        page = driver.get(dk_url)
        content = BeautifulSoup(driver.page_source)
        
        prop_cards = content.findAll("li", {"class": "game-props-card17__cell double"})
        player, odd = [], []
        for card in prop_cards:
            card = card.text.replace("−", "−-")
            card = re.split(r"−|\+", card)
            player.append(card[0])
            odd.append(card[1])
        dk_df = pd.DataFrame([player, odd]).T.rename({0: "Player", 1:"Draft King Odds"}, axis = 1)
        dk_df["Draft King Odds"] = pd.to_numeric(dk_df["Draft King Odds"])
        dk_df["Draft King Decimal Odds"] = american_to_decimal(dk_df["Draft King Odds"])
        
        return dk_df
    
    def fan_duel_webscrap(self):
        fd_url = "https://sportsbook.fanduel.com/mma?tab=ufc-fights"
        options2 = Options()
        options2.headless = True
        driver2 = webdriver.Chrome(options = options2, service=Service(ChromeDriverManager().install()))
        page2 = driver2.get(fd_url)
        content2 = BeautifulSoup(driver2.page_source)
        
        fd_players, fd_odd = [], []
        card_player = content2.findAll("span", {"class":"s y ga gb gc gd fo fp fq ft ge h cl cx gf cf"})
        card_odd = content2.findAll("span", {"class":"gh gi cl ce gr gv dl"})
        if len(card_odd) == 0:
            card_odd = content2.findAll("span", {"class":"gh gi cl ce gr gv dn"})
            
        card_title = content2.findAll("a", {"class":"t u ay w aw ax y h fw"})

        for player, odd in zip(card_player, card_odd):
            fd_players.append(player.text)
            fd_odd.append(odd.text.replace("+", ""))

        fd_df = pd.DataFrame([fd_players, fd_odd]).T.rename({0: "Player", 1:"Fan Duel Odds"}, axis = 1)
        fd_df["Fan Duel Odds"] = pd.to_numeric(fd_df["Fan Duel Odds"])
        fd_df["Fan Duel Decimal Odds"] = american_to_decimal(fd_df["Fan Duel Odds"])
        
        return fd_df, card_title
    
    def run(self):
        full_df = dk_df.merge(fd_df, on = "Player")
        matches = [x["title"] for x in card_title]
        lineup, favorable_matches = [], []
        for i, match in enumerate(matches):
            match = match.split(" v ")
            for player in match:
                try:
                    highest_odds = full_df.loc[full_df["Player"] == player, ["Draft King Decimal Odds", "Fan Duel Decimal Odds"]].max(axis=1).item()
                    best_bookie = full_df.loc[full_df["Player"] == player, ["Draft King Decimal Odds", "Fan Duel Decimal Odds"]].idxmax(axis=1).item().split()[:2]
                    best_bookie = " ".join(best_bookie)
                except:
                    highest_odds = None
                    best_bookie = None
                lineup.append([matches[i], player, highest_odds, best_bookie])

        lineup = pd.DataFrame(lineup, columns = ["Match", "Player", "Odds", "Bookie"]).dropna()

        for match in lineup["Match"].unique().tolist():
            count = lineup.loc[lineup["Match"] == match,"Match"].count()
            if count == 2:
                EV_df = lineup.loc[lineup["Match"] == match, "Odds"].tolist()
                EV = (EV_df[0]**-1) + (EV_df[1]**-1)
                print(match, round(EV,4))
                if EV < 1:
                    favorable_matches.append(match)
                else:
                    pass
            else:
                pass
                

### American to Decimal Odds Converter

In [3]:
def american_to_decimal(odds_list):
    decimal_list = []

    for odd in odds_list:
        if odd > 0:
            decimal_odd = round((odd/100) + 1,2)
            decimal_list.append(decimal_odd)
            
        elif odd < 0: 
            decimal_odd = round(1 - (100 /odd),2) 
            decimal_list.append(decimal_odd)
    return(decimal_list) 

### Draft Kings URL

In [4]:
###Avoid making multiple calls
dk_url = "https://sportsbook.draftkings.com/leagues/mma/ufc"
options = Options()
options.headless = True
driver = webdriver.Chrome(options = options, service=Service(ChromeDriverManager().install()))
page = driver.get(dk_url)
content = BeautifulSoup(driver.page_source)

prop_cards = content.findAll("li", {"class": "game-props-card17__cell double"})
player, odd = [], []
for card in prop_cards:
    card = card.text.replace("−", "−-")
    card = re.split(r"−|\+", card)
    player.append(card[0])
    odd.append(card[1])
dk_df = pd.DataFrame([player, odd]).T.rename({0: "Player", 1:"Draft King Odds"}, axis = 1)
dk_df["Draft King Odds"] = pd.to_numeric(dk_df["Draft King Odds"])
dk_df["Draft King Decimal Odds"] = american_to_decimal(dk_df["Draft King Odds"])

[WDM] - ====== WebDriver manager ======


2022-10-27 10:51:58,225 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 106.0.5249


2022-10-27 10:51:58,346 INFO Current google-chrome version is 106.0.5249


[WDM] - Get LATEST chromedriver version for 106.0.5249 google-chrome


2022-10-27 10:51:58,351 INFO Get LATEST chromedriver version for 106.0.5249 google-chrome


[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/106.0.5249.61/chromedriver_mac64.zip


2022-10-27 10:51:58,652 INFO About to download new driver from https://chromedriver.storage.googleapis.com/106.0.5249.61/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/hannibalpharathikoune/.wdm/drivers/chromedriver/mac64/106.0.5249.61]


2022-10-27 10:52:00,519 INFO Driver has been saved in cache [/Users/hannibalpharathikoune/.wdm/drivers/chromedriver/mac64/106.0.5249.61]


### Fan Duel URL

In [5]:
###Avoid making multiple calls
fd_url = "https://sportsbook.fanduel.com/mma?tab=ufc-fights"
options2 = Options()
options2.headless = True
driver2 = webdriver.Chrome(options = options2, service=Service(ChromeDriverManager().install()))
page2 = driver2.get(fd_url)
content2 = BeautifulSoup(driver2.page_source)

[WDM] - ====== WebDriver manager ======


2022-10-27 10:52:12,927 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 106.0.5249


2022-10-27 10:52:13,143 INFO Current google-chrome version is 106.0.5249


[WDM] - Get LATEST chromedriver version for 106.0.5249 google-chrome


2022-10-27 10:52:13,152 INFO Get LATEST chromedriver version for 106.0.5249 google-chrome


[WDM] - Driver [/Users/hannibalpharathikoune/.wdm/drivers/chromedriver/mac64/106.0.5249.61/chromedriver] found in cache


2022-10-27 10:52:13,353 INFO Driver [/Users/hannibalpharathikoune/.wdm/drivers/chromedriver/mac64/106.0.5249.61/chromedriver] found in cache


In [6]:
#s y ga gb gc gd fo fp fq ft ge h cl cx gf cf

#gh gi cl ce gr gv dl
#gh gi cl ce gr gv dn
fd_players, fd_odd = [], []
card_player = content2.findAll("span", {"class":"s y ga gb gc gd fo fp fq ft ge h cl cx gf cf"})
card_odd = content2.findAll("span", {"class":"gh gi cl ce gr gv dl"})
if len(card_odd) == 0:
    card_odd = content2.findAll("span", {"class":"gh gi cl ce gr gv dn"})
    
card_title = content2.findAll("a", {"class":"t ac v w y h"})

for player, odd in zip(card_player, card_odd):
    fd_players.append(player.text)
    fd_odd.append(odd.text.replace("+", ""))

fd_df = pd.DataFrame([fd_players, fd_odd]).T.rename({0: "Player", 1:"Fan Duel Odds"}, axis = 1)
fd_df["Fan Duel Odds"] = pd.to_numeric(fd_df["Fan Duel Odds"])
fd_df["Fan Duel Decimal Odds"] = american_to_decimal(fd_df["Fan Duel Odds"])

### Merge DataFrames

In [7]:
full_df = dk_df.merge(fd_df, on = "Player")

In [8]:
full_df

,Player,Draft King Odds,Draft King Decimal Odds,Fan Duel Odds,Fan Duel Decimal Odds
0,Joshua Weems,300,4.00,300,4.00
1,Chase Hooper,-295,1.34,-295,1.34
2,Steve Garcia,245,3.45,220,3.20
3,Joseph Holmes,190,2.90,200,3.00
4,Jun Yong Park,-225,1.44,-265,1.38
5,Andrei Arlovski,205,3.05,196,2.96
6,Roman Dolidze,145,2.45,144,2.44
7,Cody Durden,140,2.40,144,2.44
8,Dustin Jacoby,-170,1.59,-180,1.56
9,Khalil Rountree,145,2.45,140,2.40


### Find MMA Matches

In [9]:
matches = [x["title"] for x in card_title]
lineup, favorable_matches = [], []
margin = 10
bet = 50
for i, match in enumerate(matches):
    match = match.split(" v ") 
    for player in match:
        try:
            highest_odds = full_df.loc[full_df["Player"] == player, ["Draft King Decimal Odds", "Fan Duel Decimal Odds"]].max(axis=1).item()
            best_bookie = full_df.loc[full_df["Player"] == player, ["Draft King Decimal Odds", "Fan Duel Decimal Odds"]].idxmax(axis=1).item().split()[:2]
            best_bookie = " ".join(best_bookie)
            wage = round((margin + bet) / highest_odds,0)
        except:
            highest_odds = None
            best_bookie = None
            wage = None
        lineup.append([matches[i], player, highest_odds, best_bookie, wage])

lineup = pd.DataFrame(lineup, columns = ["Match", "Player", "Odds", "Bookie", "Wager"]).dropna()

for match in lineup["Match"].unique().tolist():
    count = lineup.loc[lineup["Match"] == match,"Match"].count()
    if count == 2:
        EV_df = lineup.loc[lineup["Match"] == match, "Odds"].tolist()
        EV = (EV_df[0]**-1) + (EV_df[1]**-1)
        print(match, round(EV,4))
        if EV < 1:
            favorable_matches.append(match)
        else:
            pass
    else:
        pass
            

Chase Hooper v Steve Garcia 1.0361
Joseph Holmes v Jun Yong Park 1.0278
Dustin Jacoby v Khalil Rountree 1.0371
Josh Fremd v Tresean Gore 1.0225
Jared Vanderaa v Waldo Cortes-Acosta 1.037
Max Griffin v Tim Means 1.0382
Calvin Kattar v Arnold Allen 1.0471
Marina Rodriguez v Amanda Lemos 1.0348
Dustin Poirier v Michael Chandler 1.0369
Israel Adesanya v Alex Pereira 1.0293
Stephen Thompson v Kevin Holland 1.0236
Jared Gordon v Paddy Pimblett 1.0378
Jan Blachowicz v Magomed Ankalaev 1.0229
Darren Till v Dricus Du Plessis 1.0226
Jiri Prochazka v Glover Teixeira 1.0396
Robert Whittaker v Paulo Costa 1.0369
Islam Makhachev v Alexander Volkanovski 1.0259


### See Favorable Matches

In [10]:
lineup.loc[(lineup["Match"].isin(favorable_matches)),:]
    

,Match,Player,Odds,Bookie,Wager
